In [2]:
from langchain_ollama import ChatOllama # type: ignore
from langchain_groq import ChatGroq # type: ignore
from langchain.prompts import ChatPromptTemplate # type: ignore
from typing import List, Dict, Any
from typing_extensions import TypedDict # type: ignore
from langchain_core.output_parsers import StrOutputParser # type: ignore
from langgraph.graph import StateGraph, START, END # type: ignore
from IPython.display import Image, display # type: ignore
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage # type: ignore
from langchain_core.output_parsers import PydanticOutputParser # type: ignore
import os
from dotenv import load_dotenv # type: ignore
# from langchain.document_loaders import PyMuPDFLoader
from typing import List, Dict, Any, Optional
import fitz # type: ignore
from pydantic import BaseModel, Field # type: ignore
load_dotenv()

False

In [3]:
os.environ["LANGSMITH_PROJECT"] = f"MineD 2025"
llm = ChatGroq(model="mixtral-8x7b-32768", api_key="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")

In [8]:
#base model to hold the metadata, and slide summaries that the llm will extract
class ResPaperText(BaseModel):
    text: str = Field(..., description="Your response")

In [28]:
instruction = """ 
You are supposed to store and recover information when asked: {extracted_text}
"""


parser = PydanticOutputParser(pydantic_object=ResPaperText)

store_template = ChatPromptTemplate(
   messages=[("system", instruction),
   ("human", "Store this information. ")],
   input_variables=["extracted_text"],
   partial_variables={"format_instructions": parser.get_format_instructions()},
)

recall_template = ChatPromptTemplate(
   messages=[("system", instruction),
   ("human", "Recall the information. ")],
   input_variables=["extracted_text"],
   partial_variables={"format_instructions": parser.get_format_instructions()},
)

structured_llm = llm.with_structured_output(ResPaperText, include_raw=True)
prompt1 = store_template.format(extracted_text="I am Mihir")
prompt2 = store_template.format(extracted_text="Who am I ?")
response1 = structured_llm.invoke(prompt1)
response2 = structured_llm.invoke(prompt2)

In [29]:
print(response2["raw"])

content='' additional_kwargs={'tool_calls': [{'id': 'call_6bgy', 'function': {'arguments': '{"text":"The user is supposed to store and recover information when asked: Who am I ?"}', 'name': 'ResPaperText'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 1082, 'total_tokens': 1187, 'completion_time': 0.161587526, 'prompt_time': 0.057210711, 'queue_time': 0.018293255000000008, 'total_time': 0.218798237}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-2c8af391-cf71-45d2-a7a3-9dcbf816c043-0' tool_calls=[{'name': 'ResPaperText', 'args': {'text': 'The user is supposed to store and recover information when asked: Who am I ?'}, 'id': 'call_6bgy', 'type': 'tool_call'}] usage_metadata={'input_tokens': 1082, 'output_tokens': 105, 'total_tokens': 1187}
